In [1]:
import re 
import os 
import string 
import unicodedata
import regex

In [17]:
table_vowel=[['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
            ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
            ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
            ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
            ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
            ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
            ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
            ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
            ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
            ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
            ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
            ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

vowel_to_ids = {}

for i in range(len(table_vowel)):
    for j in range(len(table_vowel[i]) - 1):
        vowel_to_ids[table_vowel[i][j]] = (i, j)

In [18]:
print(vowel_to_ids)

{'a': (0, 0), 'à': (0, 1), 'á': (0, 2), 'ả': (0, 3), 'ã': (0, 4), 'ạ': (0, 5), 'ă': (1, 0), 'ằ': (1, 1), 'ắ': (1, 2), 'ẳ': (1, 3), 'ẵ': (1, 4), 'ặ': (1, 5), 'â': (2, 0), 'ầ': (2, 1), 'ấ': (2, 2), 'ẩ': (2, 3), 'ẫ': (2, 4), 'ậ': (2, 5), 'e': (3, 0), 'è': (3, 1), 'é': (3, 2), 'ẻ': (3, 3), 'ẽ': (3, 4), 'ẹ': (3, 5), 'ê': (4, 0), 'ề': (4, 1), 'ế': (4, 2), 'ể': (4, 3), 'ễ': (4, 4), 'ệ': (4, 5), 'i': (5, 0), 'ì': (5, 1), 'í': (5, 2), 'ỉ': (5, 3), 'ĩ': (5, 4), 'ị': (5, 5), 'o': (6, 0), 'ò': (6, 1), 'ó': (6, 2), 'ỏ': (6, 3), 'õ': (6, 4), 'ọ': (6, 5), 'ô': (7, 0), 'ồ': (7, 1), 'ố': (7, 2), 'ổ': (7, 3), 'ỗ': (7, 4), 'ộ': (7, 5), 'ơ': (8, 0), 'ờ': (8, 1), 'ớ': (8, 2), 'ở': (8, 3), 'ỡ': (8, 4), 'ợ': (8, 5), 'u': (9, 0), 'ù': (9, 1), 'ú': (9, 2), 'ủ': (9, 3), 'ũ': (9, 4), 'ụ': (9, 5), 'ư': (10, 0), 'ừ': (10, 1), 'ứ': (10, 2), 'ử': (10, 3), 'ữ': (10, 4), 'ự': (10, 5), 'y': (11, 0), 'ỳ': (11, 1), 'ý': (11, 2), 'ỷ': (11, 3), 'ỹ': (11, 4), 'ỵ': (11, 5)}


In [30]:
def is_vietnamese_word(word):
    chars=list(word)
    vowel_index=-1
    for index,char in enumerate(chars):
        x,y=vowel_to_ids.get(char,(-1,-1))
        if x!=-1:
            if vowel_index==-1:
                vowel_index=index
            else:
                if index-vowel_index!=1:
                    #print("Not vietnamese word")
                    return False
                vowel_index=index
    #print("is vietnamese word")
    return True

In [36]:
def normalized_word(word):
    if is_vietnamese_word(word) is False:
        return word 
    
    chars=list(word)
    dau_cau=0
    vowel_indexs=[]
    qu_or_gi=False

    #remove all punctuation and save index: "khánh" ==> "khanh"
    for index,char in enumerate(chars):
        x,y=vowel_to_ids.get(char,(-1,-1))

        if x==-1:
            continue
        #check qu
        elif x==9:
            if index!=0 and chars[index-1]=='q':
                chars[index]='u'
                qu_or_gi=True
        #check gi
        elif x==5:
            if index!=0 and chars[index-1]=='g':
                chars[index]='i'
                qu_or_gi=True
        if y!=0:
            # save punctuation index 
            dau_cau=y
            chars[index]=table_vowel[x][0]
        if not qu_or_gi or index!=1:
            vowel_indexs.append(index)
    #print(chars)

    #reproduction all punctuation for word 

    #if word has at most 1 punctuation 
    if len(vowel_indexs) < 2:
        if qu_or_gi:
            if len(chars)==2:
                x,y=vowel_to_ids.get(chars[1],(-1,-1))
                chars[1]=table_vowel[x][dau_cau]
            else:
                x,y=vowel_to_ids.get(chars[2],(-1,-1))
                if x!=-1:
                    chars[2]=table_vowel[x][dau_cau]
                else: # if char[2] isn't vowel ==> char[1] is vowel
                    char[1]=table_vowel[5][dau_cau] if chars[1]=='i' else table_vowel[9][dau_cau]
            return ''.join(chars)
        return word 

    for index in vowel_indexs:
        x,y=vowel_to_ids.get(chars[index],(-1,-1))
        if x==4 or x==8:#if char='ơ' or char='ê' (excepted case )
            chars[index]=table_vowel[x][dau_cau]
            return ''.join(chars)
    
    if len(vowel_indexs)==2:
        if vowel_indexs[-1]==len(chars)-1:
            x,y=vowel_to_ids.get(chars[vowel_indexs[0]])
            chars[vowel_indexs[0]]=table_vowel[x][dau_cau]
        else:
            x,y=vowel_to_ids.get(chars[vowel_indexs[1]])
            chars[vowel_indexs[1]]=table_vowel[x][dau_cau]
    else:
        x,y=vowel_to_ids.get(chars[vowel_indexs[1]])
        chars[vowel_indexs[1]]=table_vowel[x][dau_cau]
    return ''.join(chars)           

In [37]:
normalized_word('oè')

'òe'

In [38]:
normalized_word('hoè')

'hòe'

In [40]:
normalized_word('beò')

'bèo'

In [35]:
is_vietnamese_word("Khánah")

False

In [13]:
a="Xin chào tôi là đào minh khánh"
regex.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3',a).split('/')

['Xin chào tôi là đào minh khánh']

In [42]:
file=open('/media/daominhkhanh/D:/Data/Project/ViettelProject/Data/10Label/Test_Full/Chinh tri Xa hoi/XH_NLD_T_ (8962).txt','r',encoding='utf-16')
text=file.read()

In [43]:
text

' Mạo hiểm rừng Đa Mi Cuộc hành quân khám phá thác sương mù Ẩn mình trong cánh rừng Đa Mi (Hàm Thuận Bắc, Bình Thuận) bạt ngàn là hồ thủy điện Đa Mi đẹp như một nàng công chúa, ngọn thác sương mù hùng vĩ, cao ngất, uốn lượn như con rồng bạc khổng lồ...\nXuyên rừng vào thác sương mù \nTrời đã về chiều khi chúng tôi đến được hồ thủy điện Đa Mi rộng 625ha với rất nhiều hòn đảo nhỏ đẹp như bức tranh thủy mạc dần hiện ra trong ánh hoàng hôn huyền ảo. \nNơi lưu trú đêm đầu tiên của chúng tôi là “khách sạn ngàn sao” (sân bay dã chiến của nhà máy thủy điện) với tất cả dịch vụ phải tự lo. Đến thủy điện nhưng nơi hạ trại lại không có điện. Đêm xuống bốn bề tối đen như mực. Lửa bùng lên. \nBất ngờ, Hoài Nhưỡng - bí thư chi đoàn xã Đa Mi - cùng một nhóm thanh niên xã vượt hơn 20km đường đèo vào thăm. “Lần đầu tiên có đoàn thành phố ở lại đêm đó. Hay tin bọn mình rủ nhau đi liền”, các bạn cho biết. Lời ca, tiếng hát vang theo tiếng đàn guitar chập chùng đêm rừng làm ấm lòng cả khách lẫn chủ. \nTầng

In [44]:
text=unicodedata.normalize('NFC',text)

In [45]:
text=text.lower()
words=text.split()

In [46]:
for word in 'xin'.split():
    print(regex.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word))

/xin/


In [47]:
regex.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)',r'\1/\2/\3',"20km").split('/')

['20km']

In [48]:
regex.sub('\D','1','xin chao')

'11111111'

In [53]:
words=text.split()
for word in words:
    temp=regex.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)',r'\1/\2/\3',word).split('/')
    print(word,' ',temp)

mạo   ['', 'mạo', '']
hiểm   ['', 'hiểm', '']
rừng   ['', 'rừng', '']
đa   ['', 'đa', '']
mi   ['', 'mi', '']
cuộc   ['', 'cuộc', '']
hành   ['', 'hành', '']
quân   ['', 'quân', '']
khám   ['', 'khám', '']
phá   ['', 'phá', '']
thác   ['', 'thác', '']
sương   ['', 'sương', '']
mù   ['', 'mù', '']
ẩn   ['', 'ẩn', '']
mình   ['', 'mình', '']
trong   ['', 'trong', '']
cánh   ['', 'cánh', '']
rừng   ['', 'rừng', '']
đa   ['', 'đa', '']
mi   ['', 'mi', '']
(hàm   ['(', 'hàm', '']
thuận   ['', 'thuận', '']
bắc,   ['', 'bắc', ',']
bình   ['', 'bình', '']
thuận)   ['', 'thuận', ')']
bạt   ['', 'bạt', '']
ngàn   ['', 'ngàn', '']
là   ['', 'là', '']
hồ   ['', 'hồ', '']
thủy   ['', 'thủy', '']
điện   ['', 'điện', '']
đa   ['', 'đa', '']
mi   ['', 'mi', '']
đẹp   ['', 'đẹp', '']
như   ['', 'như', '']
một   ['', 'một', '']
nàng   ['', 'nàng', '']
công   ['', 'công', '']
chúa,   ['', 'chúa', ',']
ngọn   ['', 'ngọn', '']
thác   ['', 'thác', '']
sương   ['', 'sương', '']
mù   ['', 'mù', '']
hùng   [''